In [ ]:
from pathlib import Path
import os
import copy

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input22.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input22_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

# Part 1

In [ ]:
class Brick:

    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __str__(self):
        return f"{self.start}, {self.end}"

    def compute_orientation(self):
        if self.start[0] < self.end[0]:
            self.orientation = "x"
        elif self.start[1] < self.end[1]:
            self.orientation = "y"
        elif self.start[2] < self.end[2]:
            self.orientation = "z"
        elif self.start == self.end:
            # length-1 brick
            self.orientation = "x"
        else:
            raise ValueError

    def find_all_cubes(self):
        if self.start[0] != self.end[0]:
            self.all_cubes = tuple((x, self.start[1], self.start[2]) for x in range(self.start[0], self.end[0] + 1))
        elif self.start[1] != self.end[1]:
            self.all_cubes = tuple((self.start[0], y, self.start[2]) for y in range(self.start[1], self.end[1] + 1))
        elif self.start[2] != self.end[2]:
            self.all_cubes = tuple((self.start[0], self.start[1], z) for z in range(self.start[2], self.end[2] + 1))
        else:
            # length-1 brick
            self.all_cubes = (self.start, )

In [ ]:
bricks = []
for line in data:
    start, end = line.split("~")
    start = tuple(int(x) for x in start.split(","))
    end = tuple(int(x) for x in end.split(","))
    brick = Brick(start, end)
    brick.compute_orientation()
    brick.find_all_cubes()
    bricks.append(brick)

In [ ]:
all_cubes_all_bricks = [cube for brick in bricks for cube in brick.all_cubes]
assert len(all_cubes_all_bricks) == len(set(all_cubes_all_bricks))

In [ ]:
def can_drop_one(brick, other_bricks):
    """Determines whether a brick can fall down one cube vertically"""

    if min(brick.start[2], brick.end[2]) == 0:
        # touching the ground already
        return False
    
    if brick.orientation == "z":
        # cubes_below = [[(brick.start[0], brick.start[1], brick.start[2] - 1)]
        cubes_below = [(brick.start[0], brick.start[1], brick.start[2] - 1)] + list(brick.all_cubes[:-1])
    else:
        cubes_below = [(cube[0], cube[1], cube[2] - 1) for cube in brick.all_cubes]
    
    for cube_below in cubes_below:
        if any(cube_below in brick.all_cubes for brick in other_bricks):
            return False

    return cubes_below

def find_max_drop(brick, other_bricks):
    """Determines how far a brick can fall vertically if it can."""

    z_lowest = brick.start[2]
    if z_lowest == 0:
        # touching the ground already
        return False
    
    if brick.orientation == "z":
        xy_coords = [(brick.start[0], brick.start[1])]
    else:
        xy_coords = [(cube[0], cube[1]) for cube in brick.all_cubes]

    largest_zs_other_bricks = []
    for x, y in xy_coords:
        largest_z_other_bricks = 0
        for br in other_bricks:
            for cube in br.all_cubes:
                if cube[0] == x and cube[1] == y and cube[2] < z_lowest:
                    if cube[2] > largest_z_other_bricks:
                        largest_z_other_bricks = cube[2]
                        if largest_z_other_bricks == z_lowest - 1:
                            return False
        largest_zs_other_bricks.append(largest_z_other_bricks)

    largest_z_other_bricks_overall = max(largest_zs_other_bricks)
    diff = z_lowest - largest_z_other_bricks_overall
    if diff > 1:
        new_start = (brick.start[0], brick.start[1], brick.start[2] - diff + 1)
        new_end = (brick.end[0], brick.end[1], brick.end[2] - diff + 1)
        return (new_start, new_end)
    else:
        return False

def find_supporting_bricks(brick, other_bricks):
    """Determines number of supporting bricks below"""

    if min(brick.start[2], brick.end[2]) == 0:
        # touching the ground already
        return []
    
    if brick.orientation == "z":
        # cubes_below = [[(brick.start[0], brick.start[1], brick.start[2] - 1)]
        cubes_below = [(brick.start[0], brick.start[1], brick.start[2] - 1)] + list(brick.all_cubes[:-1])
    else:
        cubes_below = [(cube[0], cube[1], cube[2] - 1) for cube in brick.all_cubes]
    
    supporting_bricks = []
    for brick in other_bricks:
        if any(cube in brick.all_cubes for cube in cubes_below):
            supporting_bricks.append(brick)

    return supporting_bricks

In [ ]:
# sort by lowest cube
bricks = sorted(bricks, key=lambda brick: min(brick.start[2], brick.end[2]))
for brick in bricks:
    print(brick)

In [ ]:
# TOO SLOW
# def let_bricks_fall(bricks, verbose=True):

#     for i, brick in enumerate(bricks):
#         if verbose:
#             print(i)

#         other_bricks = [br for br in bricks if br != brick]
#         assert len(other_bricks) == len(bricks) - 1

#         brick_current = brick
#         while True:
#             res = can_drop_one(brick_current, other_bricks)
#             if res != False:
#                 start = res[0]
#                 end = res[-1]
#                 all_cubes = res

#                 brick_current = Brick(start, end)
#                 brick_current.orientation = brick.orientation
#                 brick_current.all_cubes = all_cubes
#             else:
#                 break
            
#         bricks[i] = brick_current

#     return bricks

def let_bricks_fall_alt(bricks, verbose=True):

    for i, brick in enumerate(bricks):
        if verbose:
            print(i)

        other_bricks = [br for j, br in enumerate(bricks) if j != i]

        res = find_max_drop(brick, other_bricks)
        if res != False:
            start, end = res

            brick_new = Brick(start, end)
            brick_new.orientation = brick.orientation
            brick_new.find_all_cubes()
            
            bricks[i] = brick_new

    return bricks

In [ ]:
bricks = let_bricks_fall_alt(bricks)

In [ ]:
# sort by lowest cube
bricks = sorted(bricks, key=lambda brick: min(brick.start[2], brick.end[2]))
for brick in bricks:
    print(brick)

In [ ]:
bricks_that_cannot_be_disintegrated = []

for i, brick in enumerate(bricks):
    print(i)
    other_bricks = [br for br in bricks if br != brick]
    assert len(other_bricks) == len(bricks) - 1

    supporting_bricks_indices = find_supporting_bricks(brick, other_bricks)
    if len(supporting_bricks_indices) == 1 and supporting_bricks_indices[0] not in bricks_that_cannot_be_disintegrated:
        bricks_that_cannot_be_disintegrated.append(supporting_bricks_indices[0])

In [ ]:
len(bricks)

In [ ]:
len(bricks_that_cannot_be_disintegrated)

In [ ]:
len(bricks) - len(bricks_that_cannot_be_disintegrated)

# Part 2

Slow approach:

In [ ]:
# This is very slow

num_dislodged_bricks_total = 0

bricks_original = copy.deepcopy(bricks)
for i in range(len(bricks)):
    print(f"Excluding brick {i}")
    bricks_excl_one = copy.deepcopy(bricks_original[:i] + bricks_original[(i + 1):])
    bricks_excl_one_original = copy.deepcopy(bricks_excl_one)

    bricks_result = let_bricks_fall_alt(bricks_excl_one, verbose=False)

    num_dislodged_bricks = sum(1 for brick_original, brick_new in zip(bricks_excl_one_original, bricks_result) if brick_original.all_cubes != brick_new.all_cubes)
    print(f"{num_dislodged_bricks = }")
    num_dislodged_bricks_total += num_dislodged_bricks

In [ ]:
print(num_dislodged_bricks_total)

It is faster to store for each brick which base bricks it is supported by. Then keep a queue of falling bricks, initialised with the excluded brick. 

See
https://github.com/derailed-dash/Advent-of-Code/blob/master/src/AoC_2023/Dazbo's_Advent_of_Code_2023.ipynb

In [ ]:
def find_supporting_bricks_indices(brick_idx, bricks):
    """Determines number of supporting bricks below"""

    brick = bricks[brick_idx]

    if min(brick.start[2], brick.end[2]) == 0:
        # touching the ground already
        return []
    
    if brick.orientation == "z":
        # cubes_below = [[(brick.start[0], brick.start[1], brick.start[2] - 1)]
        cubes_below = [(brick.start[0], brick.start[1], brick.start[2] - 1)] + list(brick.all_cubes[:-1])
    else:
        cubes_below = [(cube[0], cube[1], cube[2] - 1) for cube in brick.all_cubes]
    
    supporting_bricks_indices = []
    for j, brick in enumerate(bricks):
        if j == brick_idx:
            continue
        if any(cube in brick.all_cubes for cube in cubes_below):
            supporting_bricks_indices.append(j)

    return supporting_bricks_indices

In [ ]:
num_bricks = len(bricks)

supporting_bricks_map = {}
for i in range(num_bricks):
    supporting_bricks_indices = find_supporting_bricks_indices(i, bricks)
    supporting_bricks_map[i] = supporting_bricks_indices

In [ ]:
num_dislodged_bricks_total = 0

for i in range(num_bricks):
    print(f"Excluding brick {i}")

    supporting_bricks_map_copy = copy.deepcopy(supporting_bricks_map)

    falling_brick_idx_queue = [i]
    num_dislodged_bricks = 0
    while len(falling_brick_idx_queue) > 0:
        current_brick_idx = falling_brick_idx_queue.pop()

        for j in range(num_bricks):
            if current_brick_idx == j:
                continue

            if current_brick_idx in supporting_bricks_map_copy[j]:
                supporting_bricks_map_copy[j].remove(current_brick_idx)
                
                if len(supporting_bricks_map_copy[j]) == 0 and j not in falling_brick_idx_queue:
                    falling_brick_idx_queue.append(j)
                    num_dislodged_bricks += 1

    print(f"{num_dislodged_bricks = }")
    num_dislodged_bricks_total += num_dislodged_bricks

In [ ]:
print(num_dislodged_bricks_total)